In [1]:
import os, sys
import numpy as np
import dask
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)
#print(cluster.job_script())
w = cluster.start_workers(20)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [5]:
client

Client Scheduler: tcp://10.148.1.31:53370 Dashboard: http://10.148.1.31:8787/status,Cluster Workers: 40 Cores: 160 Memory: 2000.00 GB


--- 

# standard data layout: (face, j, i), 1 file per time (for movies)

In [1]:
V = ['SSU', 'SSV']

'SSV'

In [4]:
V[0]

'SSU'

In [ ]:
V = ['SSV']
U = ['SSU']

Nt = 1
out_dir = '/home1/scratch/xyu/mit_zarr/'  
print(out_dir)

fsize_bound = 13*60*1e6

for v in V:
    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
    #it = np.arange(time.size/Nt-1).astype(int)*Nt
    it = np.arange(720).astype(int)*Nt # tmp
    assert it[-1]+Nt<time.size
    #
    p = 'C'
    vdrop = ['XC','YC','Depth','rA']
    if v is 'SSU':
        p = 'W'
        vdrop = ['dxC','dyG','rAw']        
    elif v is 'SSV':
        p = 'S'
        vdrop = ['dxG','dyC','rAs']        
    #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    ds_u = get_compressed_data(U, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)

    
    
    #ds = ds.chunk({'face': 1})
    #
    for i, t in enumerate(it):
        #
        file_out = out_dir+'/%s_t%04d.nc'%(v,i)
        if not os.path.isfile(file_out) or os.path.getsize(file_out) < fsize_bound:            
            dv = ds[v].isel(time=slice(t,t+Nt)) 
            
            
            # should store grid data independantly in a single file
            dv = dv.drop(vdrop).to_dataset()
            #
            #print(dv)
            #
            while True:
                try:
                    #print(dv)
                    %time dv.to_netcdf(file_out, mode='w')                    
                except:
                    print('Failure')
                if os.path.isfile(file_out) and os.path.getsize(file_out) > fsize_bound:
                    #
                    print('i=%d, iter=%d'%(i, iters[i].values))
                    break
        else:
            print('i=%d, iter=%d - allready processed'%(i, iters[i].values))

In [12]:
cluster.scheduler.close()

<Future finished result=None>